In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
# import pydicom
import warnings

from PIL import Image
import cv2

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt
import random
paddingSize= 0

warnings.filterwarnings("ignore")

In [ ]:
# unzip model

import zipfile
import os

zip_file_path = '/kaggle/input/faster-resnet-p12/_output_.zip'
extract_to = '/kaggle/working/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

In [ ]:
# Load Model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torch.load('/kaggle/working/resnet_bm_p12.pth')#,map_location=torch.device('cpu'))
model.to(device)
model.eval()
# cpu_device = torch.device("cpu")

In [ ]:
import cv2
import torch
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms.functional import to_tensor
from PIL import Image

# Load the pre-trained Faster R-CNN model
# model = fasterrcnn_resnet50_fpn(pretrained=True)
# model.eval()

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Open the input video file
cap = cv2.VideoCapture('/kaggle/input/drone-detection-test-videos-2/drone_video_19.mp4')

# Get the video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('video_19.avi', fourcc, fps, (width, height))

# Define the transformation to apply to each frame
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    tensor_image = to_tensor(frame).unsqueeze(0).to(device)

    # Convert the frame to a PIL image and apply the transformation
#     pil_image = Image.fromarray(frame)
#     tensor_image = transform(pil_image).unsqueeze(0).to(device)

    # Perform object detection
    with torch.no_grad():
        output = model(tensor_image)

    # Draw bounding boxes on the frame
    for box in output[0]['boxes']:
        box = [int(coord) for coord in box.cpu().numpy()]
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)

    # Write the frame with bounding boxes to the output video
    out.write(frame)

# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()
# cv2.destroyAllWindows()
